In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import keras
import tensorflow as tf
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras import backend as K
from tensorflow.keras.layers import LayerNormalization
from tensorflow.keras.optimizers import Adam, RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.applications.resnet50 import ResNet50
from keras import backend as K 

In [ ]:
os.makedirs('/content/augment/')
import albumentations as A

In [ ]:
transform = A.Compose([A.HorizontalFlip(p=0.5),])
transform2=A.Compose([A.Rotate(p=0.2),])
transform3=A.Compose([A.GaussNoise(var_limit=(10, 50),p=0.05),])

In [ ]:
import cv2
c=0
f=0
for i in os.listdir('/content/cancerdata/train/malignant'):
  if c<81:
    image = cv2.imread("/content/cancerdata/train/malignant/"+str(i))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    transformed = transform(image=image)
    transformed_image = transformed["image"]
    f=f+1
    transformed_image= Image.fromarray(transformed_image)
    transformed_image.save('/content/augment/'+str(f) + "aug" + ".jpg")

    transformed2 = transform2(image=image)
    transformed_image = transformed2["image"]
    f=f+1
    transformed_image= Image.fromarray(transformed_image)
    transformed_image.save('/content/augment/'+str(f) + "aug" + ".jpg")

    transformed3 = transform3(image=image)
    transformed_image = transformed3["image"]
    f=f+1
    transformed_image= Image.fromarray(transformed_image)
    transformed_image.save('/content/augment/'+str(f) + "aug" + ".jpg")
  c=c+1

In [ ]:
import shutil
import os
    
source_dir = '/content/augment'
target_dir = '/content/cancerdata/train/malignant/'
    
file_names = os.listdir(source_dir)
    
for file_name in file_names:
    shutil.move(os.path.join(source_dir, file_name), target_dir)

In [ ]:
import matplotlib.pyplot as plt
def plot(imgs, with_orig=True, row_title=None, **imshow_kwargs):
    if not isinstance(imgs[0], list):
        # Make a 2d grid even if there's just 1 row
        imgs = [imgs]

    num_rows = len(imgs)
    num_cols = len(imgs[0]) + with_orig
    fig, axs = plt.subplots(nrows=num_rows, ncols=num_cols, squeeze=False)
    for row_idx, row in enumerate(imgs):
        row = [orig_img] + row if with_orig else row
        for col_idx, img in enumerate(row):
            ax = axs[row_idx, col_idx]
            ax.imshow(np.asarray(img), **imshow_kwargs)
            ax.set(xticklabels=[], yticklabels=[], xticks=[], yticks=[])

    if with_orig:
        axs[0, 0].set(title='Original image')
        axs[0, 0].title.set_size(8)
    if row_title is not None:
        for row_idx in range(num_rows):
            axs[row_idx, 0].set(ylabel=row_title[row_idx])

    plt.tight_layout()

In [ ]:
def save_imgs(jitted_imgs,f,with_orig=True):
  if not isinstance(jitted_imgs[0], list):
        # Make a 2d grid even if there's just 1 row
        jitted_imgs = [jitted_imgs]
  num_rows = len(jitted_imgs)
  num_cols = len(jitted_imgs[0]) + with_orig
  fig, axs = plt.subplots(nrows=num_rows, ncols=num_cols, squeeze=False)
  for row_idx, row in enumerate(jitted_imgs):
      row = [orig_img] + row if with_orig else row
      for col_idx, img in enumerate(row):
        res=Image.fromarray(np.asarray(img))
        res.save('/content/augment/'+str(f) + "aug" + ".jpg")
        f=f+1

In [ ]:
from torchvision import transforms
from torchvision.transforms import Compose as C
import torchvision
import numpy as np
from PIL import Image
f=0
for i in os.listdir('/content/cancerdata/train/benign/'):
  orig_img=Image.open('/content/cancerdata/train/benign/'+i)
  color_aug = torchvision.transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.1, hue=0.3)
  jitted_imgs = [color_aug(orig_img) for _ in range(4)]
  save_imgs(jitted_imgs,f)
  f=f+1000

In [ ]:
def norm():
  
  normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
  image_batch, labels_batch = next(iter(normalized_ds))
  first_image = image_batch[0]

  print(np.min(first_image), np.max(first_image))